Import modules

In [0]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

데이터 불러오기, 결측치 제거, 변수 선택

In [161]:
# 데이터 불러오기

data = pd.read_csv("clogit_df.csv",encoding='utf-8')
print("전체 데이터 개수 : ",len(data))

# 결측치 제거

data = data.dropna(subset=['distance','record',
                           'code','lane','age','jockey_w','dandivi','yeondivi',
                           'cure_in_1m','weight_diff', 'raw_weight','weight_added','jockey',
                           'prev1_velo','prev2_velo','sex1','sex2'])

print("결측치 제거 후 개수 : ",len(data))


# distance별로 데이터 나누기

gb_distance = data.groupby(['distance'])
gb_distance=[gb_distance.get_group(x) for x in gb_distance.groups]

# distance list
distance_list = list(set(data['distance']))
distance_list.sort()

for i in range(len(gb_distance)):
  print(distance_list[i] ,"m 거리에서 데이터 개수 : ",len(gb_distance[i]))




# 데이터 가장 많은 Dataframe 선택  

data=gb_distance[2]


# X,y 데이터 나누기

y = data['record']
X = data.loc[:, ['code','lane','age','jockey_w','dandivi','yeondivi',
                 'cure_in_1m','weight_diff', 'raw_weight','weight_added','jockey',
                 'prev1_velo','prev2_velo','sex1','sex2']]


# 
X = X.as_matrix()
y = y.as_matrix()


전체 데이터 개수 :  56576
결측치 제거 후 개수 :  56531
1000.0 m 거리에서 데이터 개수 :  6396
1100.0 m 거리에서 데이터 개수 :  479
1200.0 m 거리에서 데이터 개수 :  13562
1300.0 m 거리에서 데이터 개수 :  12916
1400.0 m 거리에서 데이터 개수 :  9912
1700.0 m 거리에서 데이터 개수 :  5609
1800.0 m 거리에서 데이터 개수 :  5669
1900.0 m 거리에서 데이터 개수 :  1083
2000.0 m 거리에서 데이터 개수 :  850
2300.0 m 거리에서 데이터 개수 :  55


In [0]:
# 설정하는 부분

learning_rate = 0.01
training_epochs = 200
batch_size = 128


In [163]:
# Train , Test Split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

train_size = len(X_train)
print("train_size : ",train_size)

size_x = len(X_train[0])
print("size_x : ",size_x)



train_size :  9493
size_x :  15
1 번째 변수의 X_mean: 32375.601 , X_std: 3026.472
2 번째 변수의 X_mean: 6.321 , X_std: 3.498
3 번째 변수의 X_mean: 3.552 , X_std: 1.003
4 번째 변수의 X_mean: 53.538 , X_std: 2.020
5 번째 변수의 X_mean: 43.449 , X_std: 61.989
6 번째 변수의 X_mean: 6.844 , X_std: 8.195
7 번째 변수의 X_mean: 0.208 , X_std: 0.406
8 번째 변수의 X_mean: -0.037 , X_std: 7.229
9 번째 변수의 X_mean: 470.189 , X_std: 27.077
10 번째 변수의 X_mean: -0.037 , X_std: 7.229
11 번째 변수의 X_mean: 80435.642 , X_std: 1380.771
12 번째 변수의 X_mean: 15.625 , X_std: 0.411
13 번째 변수의 X_mean: 15.614 , X_std: 0.413
14 번째 변수의 X_mean: 0.313 , X_std: 0.464
15 번째 변수의 X_mean: 0.475 , X_std: 0.499


In [0]:
# Normalize 

X_mean = []
X_std = []

for i in range(X_train.shape[1]):
  X_mean.append(np.mean(X_train[:,i]))
  X_std.append(np.std(X_train[:,i]))
  print(i+1,"번째 변수의 X_mean:",'{:.3f}'.format(X_mean[i]),", X_std:",'{:.3f}'.format(X_std[i]))
  X_train[:,i] = (X_train[:,i]-X_mean[i])/X_std[i]
  X_test[:,i] = (X_test[:,i]-X_mean[i])/X_std[i]

  



In [0]:
# Tensorflow

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, size_x])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)
phase = tf.placeholder(tf.bool, name='phase')

# First Layer

W1 = tf.get_variable(shape=[size_x, 128], name='weight1', initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([128]))
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(layer1, keep_prob=keep_prob)
layer1 = tf.contrib.layers.batch_norm(layer1, center=True, scale=True, 
                                          is_training=phase)

# Second Layer

W2 = tf.get_variable(shape=[128, 64], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(layer2, keep_prob=keep_prob)
layer2 = tf.contrib.layers.batch_norm(layer2, center=True, scale=True, 
                                          is_training=phase)
# Third Layer

W3 = tf.get_variable(shape=[64, 32], name='weight3', initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([32]))
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(layer3, keep_prob=keep_prob)
layer3 = tf.contrib.layers.batch_norm(layer3, center=True, scale=True, 
                                          is_training=phase)
# Predict

W4 = tf.get_variable(shape=[32, 1], name='weight4', initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([1]))
hypothesis = tf.matmul(layer3, W4) + b4
hypothesis = tf.contrib.layers.batch_norm(hypothesis, center=True, scale=True, 
                                          is_training=phase)



In [0]:
# Cost 계산

cost = tf.reduce_mean(tf.square(hypothesis - Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



In [0]:
# 계산 시작

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(train_size / batch_size)

    for i in range(total_batch):
        # batch_size로 X,y 개수 나눠주기
        batch_xs = X_train[i*batch_size:(i+1)*batch_size]
        batch_ys = y_train[i*batch_size:(i+1)*batch_size].reshape(batch_size,1)
        
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob:0.7,phase:True}
        c, _ = sess.run([cost, train], feed_dict=feed_dict)
        avg_cost += c/total_batch
        
    # 20 epoch마다 cost
    
    if (epoch+1) % 20==0:
      print('Epoch:', '%04d' % (epoch+1), 'cost =', '{:.5f}'.format(avg_cost))
      


In [0]:
# Test set 계산

y_test = y_test.reshape(-1,1)
mse = tf.reduce_mean(tf.square(hypothesis - Y))**0.5
print('Cost of test set:', sess.run(mse, feed_dict={
      X: X_test, Y: y_test, keep_prob:1, phase:False}))

Epoch: 0020 cost = 4035.11239
Epoch: 0040 cost = 2534.52882
Epoch: 0060 cost = 1443.26181
Epoch: 0080 cost = 702.49196
Epoch: 0100 cost = 266.72007
Epoch: 0120 cost = 75.84534
Epoch: 0140 cost = 28.19379
Epoch: 0160 cost = 24.69719
Epoch: 0180 cost = 24.23979
Epoch: 0200 cost = 24.56237
Epoch: 0220 cost = 25.15062
Epoch: 0240 cost = 24.84709
Epoch: 0260 cost = 24.69363
Epoch: 0280 cost = 24.89237
Epoch: 0300 cost = 26.09980


Cost of test set: 146930.39
